# bert-financial-sentiment-launcher

----

In [ ]:
# 导入必要的库，包括 os 用于操作文件系统，numpy 用于处理数组，pandas 用于处理数据表格，以及 sagemaker 用于使用 Amazon SageMaker 服务。
import os 

import numpy as np 

import pandas as pd 

import sagemaker 

# 创建一个 Amazon SageMaker 会话。
sagemaker_session = sagemaker.Session() 

# 设置存储桶名称。
bucket = "<bucket name>" 

# 设置存储桶中文件的前缀。
prefix = "sagemaker/pytorch-bert-financetext" 

# 获取执行角色。
role = sagemaker.get_execution_role() 

# 将训练数据上传到指定的存储桶中，文件路径为 './data/train.csv'。
inputs_train = sagemaker_session.upload_data("./data/train.csv", bucket=bucket, key_prefix=prefix) 

# 将测试数据上传到指定的存储桶中，文件路径为 './data/test.csv'。
inputs_test = sagemaker_session.upload_data("./data/test.csv", bucket=bucket, key_prefix=prefix) 

**解释：**

这段代码使用 Amazon SageMaker 服务将本地训练数据和测试数据上传到指定的存储桶中，为后续的模型训练和评估做准备。

1. **导入库：**  导入必要的库，包括 `os` 用于操作文件系统，`numpy` 用于处理数组，`pandas` 用于处理数据表格，以及 `sagemaker` 用于使用 Amazon SageMaker 服务。
2. **创建 SageMaker 会话：** 创建一个 Amazon SageMaker 会话，使用 `sagemaker.Session()`。SageMaker 会话对象提供了一系列方法，用于管理和运行 SageMaker 任务。
3. **设置存储桶信息：** 设置存储桶名称 `bucket` 和文件前缀 `prefix`。`bucket` 是存储数据的存储桶名称，`prefix` 是存储桶中目录的路径。
4. **获取执行角色：** 获取执行角色 `role`。执行角色是用于授权 SageMaker 任务访问其他 AWS 服务的权限。
5. **上传训练数据：** 使用 `sagemaker_session.upload_data()` 方法将本地训练数据文件 `./data/train.csv` 上传到指定的存储桶中。
6. **上传测试数据：** 使用 `sagemaker_session.upload_data()` 方法将本地测试数据文件 `./data/test.csv` 上传到指定的存储桶中。

这段代码展示了如何使用 Amazon SageMaker 将本地数据上传到 S3 存储桶中，以便后续使用 SageMaker 训练和评估模型。


----

In [ ]:
# 导入 PyTorch 类的 SageMaker estimator。
from sagemaker.pytorch import PyTorch 

# 设置模型输出路径。
output_path = f"s3://{bucket}/{prefix}" 

# 创建一个 PyTorch estimator，用于在 SageMaker 上训练模型。
estimator = PyTorch( 

    # 设置模型训练脚本入口。
    entry_point="train.py", 

    # 设置存放模型代码的目录。
    source_dir="code", 

    # 设置 SageMaker 任务的执行角色。
    role=role, 

    # 设置 PyTorch 框架版本。
    framework_version="1.6", 

    # 设置 Python 版本。
    py_version="py3", 

    # 设置训练实例的数量。
    instance_count=1,   

    # 设置训练实例的类型。
    instance_type="ml.p3.2xlarge", 

    # 设置模型输出路径，模型训练完成后模型数据将保存到该路径。
    output_path=output_path, 

    # 设置模型训练超参数。
    hyperparameters={ 

        # 设置训练的 epoch 数。
        "epochs": 4, 

        # 设置学习率。
        "lr" : 5e-5, 

        # 设置标签的数量。
        "num_labels": 3, 

        # 设置训练数据的文件名。
        "train_file": "train.csv", 

        # 设置测试数据的文件名。
        "test_file" : "test.csv", 

        # 设置最大序列长度。
        "MAX_LEN" : 315, 

        # 设置训练数据的 batch 大小。
        "batch-size" : 16, 

        # 设置测试数据的 batch 大小。
        "test-batch-size" : 10 

    } 

) 

# 使用 fit 方法训练模型，将训练数据和测试数据传入。
estimator.fit({"training": inputs_train, "testing": inputs_test}) 

**解释：**

这段代码使用 Amazon SageMaker 的 PyTorch estimator 在云端训练一个 PyTorch 模型。

1. **创建 PyTorch Estimator：** 使用 `PyTorch` 类创建一个用于训练 PyTorch 模型的 estimator，并设置模型训练的各种参数：
    * `entry_point`： 模型训练脚本的入口文件，这里设置为 `train.py`。
    * `source_dir`： 存放模型代码的目录，这里设置为 `code`。
    * `role`： SageMaker 任务的执行角色。
    * `framework_version`： PyTorch 框架版本，这里设置为 `1.6`。
    * `py_version`： Python 版本，这里设置为 `py3`。
    * `instance_count`： 训练实例的数量，这里设置为 1，表示使用 1 个实例进行训练。
    * `instance_type`： 训练实例的类型，这里设置为 `ml.p3.2xlarge`，表示使用性能较高的实例进行训练。
    * `output_path`： 模型输出路径，模型训练完成后模型数据将保存到该路径。
    * `hyperparameters`： 模型训练超参数，包含以下参数：
      * `epochs`： 训练的 epoch 数。
      * `lr`： 学习率。
      * `num_labels`： 标签的数量。
      * `train_file`： 训练数据的文件名。
      * `test_file`： 测试数据的文件名。
      * `MAX_LEN`： 最大序列长度。
      * `batch-size`： 训练数据的 batch 大小。
      * `test-batch-size`： 测试数据的 batch 大小。

2. **启动训练：** 使用 `estimator.fit` 方法启动模型训练，并将训练数据和测试数据传入，训练数据存储在 `inputs_train` 中，测试数据存储在 `inputs_test` 中。

这段代码展示了如何使用 Amazon SageMaker 的 PyTorch estimator 训练深度学习模型，并利用云计算资源加速模型训练过程。


----

In [ ]:
# 导入 PyTorch 模型类的 SageMaker estimator。
from sagemaker.pytorch.model import PyTorchModel 

# 获取训练好的模型数据，存储在 model_data 变量中。
model_data = estimator.model_data 

# 创建一个 PyTorch 模型，用于部署在 SageMaker 上，并设置模型参数。
pytorch_model = PyTorchModel(model_data=model_data, 

                             # 设置 SageMaker 任务的执行角色。
                             role=role, 

                             # 设置 PyTorch 框架版本。
                             framework_version="1.6", 

                             # 设置存放模型代码的目录。
                             source_dir="code", 

                             # 设置 Python 版本。
                             py_version="py3", 

                             # 设置模型推理脚本的入口。
                             entry_point="inference.py") 

# 部署模型，并设置实例数量和类型，将模型部署成一个可访问的端点，供用户进行预测调用。
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge") 


**解释：**

这段代码用于将训练好的 PyTorch 模型部署到 Amazon SageMaker 上，使其可以被用户访问并进行预测。

1. **创建 PyTorch 模型：**  使用 `PyTorchModel` 类创建一个用于部署的 PyTorch 模型，并设置模型参数：
    * `model_data`： 训练好的模型数据。
    * `role`： SageMaker 任务的执行角色。
    * `framework_version`： PyTorch 框架版本。
    * `source_dir`： 存放模型代码的目录。
    * `py_version`： Python 版本。
    * `entry_point`： 模型预测脚本的入口文件，这里设置为 `inference.py`。

2. **部署模型：** 使用 `deploy` 方法将 PyTorch 模型部署到 SageMaker 上。
    * `initial_instance_count`： 部署的实例数量，这里设置为 1，表示使用 1 个实例进行部署。
    * `instance_type`： 部署的实例类型，这里设置为 `ml.m4.xlarge`，表示使用性能较高的实例进行部署。

这段代码展示了如何使用 Amazon SageMaker 的 `PyTorchModel` 将训练好的模型部署为可以访问的端点，供用户进行预测。

**代码逻辑：**

这段代码通过加载训练好的模型数据，并使用 `PyTorchModel` 类创建了用于部署的 PyTorch 模型。然后调用 `deploy` 方法将模型部署到 Amazon SageMaker 上，并设置部署的参数，包括实例数量和实例类型。最后，代码将部署好的模型的预测器对象存储在 `predictor` 变量中，方便后续使用预测器对新数据进行预测。 


----

In [ ]:
# 设置预测器的序列化器为 JSONSerializer，用于将预测请求序列化为 JSON 格式。
predictor.serializer = sagemaker.serializers.JSONSerializer() 

# 设置预测器的反序列化器为 JSONDeserializer，用于将预测结果反序列化为 JSON 格式。
predictor.deserializer = sagemaker.deserializers.JSONDeserializer() 

# 使用部署好的模型进行预测，并将预测结果存储在 result 变量中。
result = predictor.predict("The market is doing better than last year") 

# 获取预测结果中概率最大的类别索引，并打印预测结果。
print("预测类别: ", np.argmax(result, axis=1))  

**解释：**

这段代码使用部署好的模型对输入文本进行预测，并打印预测结果。

1. **设置序列化器和反序列化器：**  使用 `JSONSerializer` 将预测请求序列化为 JSON 格式，使用 `JSONDeserializer` 将预测结果反序列化为 JSON 格式。
2. **进行预测：**  使用 `predictor.predict` 方法对输入文本进行预测。
3. **获取预测结果：**  获取预测结果中概率最大的类别索引，并打印预测结果。

这段代码展示了如何使用部署好的 SageMaker 模型进行预测，并使用 `JSONSerializer` 和 `JSONDeserializer` 将预测请求和结果序列化为 JSON 格式。


----

In [ ]:
# 使用 predictor.delete_endpoint() 方法删除部署的模型端点。
predictor.delete_endpoint()

**解释：**

这段代码使用 `predictor.delete_endpoint()` 方法删除之前部署的模型端点。

* **`predictor.delete_endpoint()`：**  是 SageMaker 提供的用于删除模型端点的函数。删除端点后，模型的预测服务将会停止提供服务，并且对应的实例将会被释放。

通过使用 `delete_endpoint()` 方法，开发者可以删除不再需要的模型端点，释放对应的资源。

**代码逻辑：**

这段代码简单地调用了 `delete_endpoint()` 方法，从而删除模型端的点。


----